In [ ]:
from transformers import pipeline

class XLMR:
    def __init__(self):
        self.model_name = 'xlmr'
        self.nlp = pipeline(task ="feature-extraction", model = 'xlm-roberta-base', tokenizer='xlm-roberta-base', framework='pt', device=0)

    def GetFeatures(self, sentences=None):
        if self.model_name == 'xlmr':
            features = self.nlp(sentences, truncation=True) 
            # featurelist=list()
            # for i in features:
            #    featurelist.append(i[0][0])
            # self.features=pd.DataFrame(featurelist)        
        return features[0][0]

In [ ]:
xlmr = XLMR()

In [ ]:
import torch
import torch.nn as nn

class CustomBERTModel(nn.Module):
    def __init__(self):
          super(CustomBERTModel, self).__init__()
     
          self.linear1 = nn.Linear(1536, 768)
          self.linear2 = nn.Linear(768, 300)
          self.linear3 = nn.Linear(1068, 768)
          self.linear4 = nn.Linear(768,2)
          self.outputlayer = nn.Softmax(dim=1)

    def concatenate(self, tensor1, tensor2):
      return torch.cat((tensor1, tensor2), 1)
      
    def forward(self, post=None, context=None, text=None, only_post = False, post_context=False, post_context_text=False):

      if only_post:
        input_post_embeddings = post
        
        outputs = input_post_embeddings

      if post_context:

        input_post_embeddings = post

        input_context_embeddings = context

        post_context = self.concatenate(input_post_embeddings, input_context_embeddings)

        linear1_output = self.linear1(post_context)

        outputs = linear1_output

      if post_context_text:

        input_post_embeddings = post

        input_context_embeddings = context

        input_text_embeddings = text

        context_text = self.concatenate(input_context_embeddings, input_post_embeddings)

        linear1_output = self.linear1(context_text)

        linear2_output = self.linear2(linear1_output)

        post_context_text = self.concatenate(input_text_embeddings, linear2_output)

        linear3_output = self.linear3(post_context_text)

        outputs = linear3_output

      return outputs

In [ ]:
model = CustomBERTModel()
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model.load_state_dict(torch.load("rev_xlmr_model_adamw.pth"))

In [ ]:
model = model.to(device)

In [ ]:
import pandas as pd
train = pd.read_pickle("train_hierarchial.pkl")

In [ ]:
X = []
y = []


for post, context,text, label in zip(train["post"], train["context"], train["text"], train["label"]):

    if post!="" and context=="" and text=="":

        input_post=xlmr.GetFeatures(post)
        input_post = torch.FloatTensor([input_post]).to(device)

        X.append(model(input_post, only_post=True))
        y.append(label)
    

    if post!="" and context!="" and text=="":

        input_post=xlmr.GetFeatures(post)
        input_post = torch.FloatTensor([input_post]).to(device)

        input_context=xlmr.GetFeatures(context)
        input_context = torch.FloatTensor([input_context]).to(device)

        X.append(model(input_post, input_context,post_context=True))
        y.append(label)


    if post!="" and context!="" and text!="":

        input_post=xlmr.GetFeatures(post)
        input_post = torch.Tensor([input_post]).to(device)

        input_context=xlmr.GetFeatures(context)
        input_context = torch.Tensor([input_context]).to(device)

        input_text=xlmr.GetFeatures(text)
        input_text = torch.Tensor([input_text]).to(device)

        X.append(model(input_post, input_context,input_text, post_context_text=True))
        y.append(label)

In [ ]:
x = pd.DataFrame([feat.to("cpu").tolist()[0] for feat in X][:])

In [ ]:
import numpy as np

train_X = x.values
train_y = np.array(y)

In [ ]:
test = pd.read_pickle("test_hierarchial.pkl")

In [ ]:
test_X = []
test_y = []


for post, context,text, label in zip(test["post"], test["context"], test["text"], test["label"]):

    if post!="" and context=="" and text=="":

        input_post=xlmr.GetFeatures(post)
        input_post = torch.FloatTensor([input_post]).to(device)

        test_X.append(model(input_post, only_post=True))
        test_y.append(label)
    

    if post!="" and context!="" and text=="":

        input_post=xlmr.GetFeatures(post)
        input_post = torch.FloatTensor([input_post]).to(device)

        input_context=xlmr.GetFeatures(context)
        input_context = torch.FloatTensor([input_context]).to(device)

        test_X.append(model(input_post, input_context,post_context=True))
        test_y.append(label)


    if post!="" and context!="" and text!="":

        input_post=xlmr.GetFeatures(post)
        input_post = torch.Tensor([input_post]).to(device)

        input_context=xlmr.GetFeatures(context)
        input_context = torch.Tensor([input_context]).to(device)

        input_text=xlmr.GetFeatures(text)
        input_text = torch.Tensor([input_text]).to(device)

        test_X.append(model(input_post, input_context,input_text, post_context_text=True))
        test_y.append(label)

In [ ]:
test_x = pd.DataFrame([feat.to("cpu").tolist()[0] for feat in test_X][:])

In [ ]:
test_X = test_x.values
test_y = np.array(test_y)

In [ ]:
params_xgboost = {"n_estimators": 500, "learning_rate": 1, "max_depth": 2,
                  "objective": 'multi:softmax', "num_class": 4}
params_lr = {"multi_class": 'multinomial'}
params_rf = {"n_estimators": 500, "criterion": 'entropy', "random_state": 0}

In [ ]:
import pickle
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

class Classifiers:
    def __init__(self, train_X, train_y, test_X, test_y):
        self.Createstaticsplit(train_X, train_y, test_X, test_y)
        self.accuracy = list()
        self.f1score = list()

    def MajorityVotingClassifier(self, num_class):
        _, _, y_xgboost = self.XGBClassifier(num_class)
        _, _, y_rf = self.RandomForestClassifier()
        _, _, y_lr = self.LogisticRegression()
        y_pred = list()
        for i in range(len(y_xgboost)):
            preds = list()
            preds.append(y_xgboost[i])
            preds.append(y_rf[i])
            preds.append(y_lr[i])
            y_pred.append(max(set(preds), key=preds.count))

        with open('p2_predictions.pkl', 'wb') as f:
          pickle.dump(y_pred, f)

        return accuracy_score(
            self.test_labels, y_pred), f1_score(
            self.test_labels, y_pred, average='weighted')

    def XGBClassifier(self, num_class):
        classifier = XGBClassifier(**params_xgboost)
        classifier.fit(self.train_features, self.train_labels)
        y_pred = classifier.predict(self.test_features)
        return accuracy_score(self.test_labels, y_pred), f1_score(self.test_labels, y_pred, average='weighted'), y_pred

    def RandomForestClassifier(self):
        classifier = RandomForestClassifier(**params_rf)
        classifier.fit(self.train_features, self.train_labels)
        y_pred = classifier.predict(self.test_features)
        return accuracy_score(self.test_labels, y_pred), f1_score(self.test_labels, y_pred, average='weighted'), y_pred

    def Createstaticsplit(self, train_X, train_y, test_X, test_y):
        self.train_features = train_X
        self.train_labels = train_y
        self.test_features = test_X
        self.test_labels = test_y

    def LogisticRegression(self):
        lr_clf = LogisticRegression(**params_lr)
        lr_clf.fit(self.train_features, self.train_labels)
        y_pred = lr_clf.predict(self.test_features)
        return accuracy_score(self.test_labels, y_pred), f1_score(self.test_labels, y_pred, average='weighted'), y_pred

In [ ]:
handler = Classifiers(train_X, train_y, test_X, test_y)

accuracies,f1_scores = handler.MajorityVotingClassifier(2)

print("Accuracy: {}, F1 Score: {}".format(accuracies, f1_scores))

In [ ]:
classifier = RandomForestClassifier(**params_rf)
classifier.fit(train_X, train_y)

In [ ]:
y_pred = classifier.predict(test_x)
accuracy_score(test_y, y_pred)

In [ ]:
import sklearn
from lime import lime_text
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(lowercase=False)
c = make_pipeline(vectorizer, classifier)   

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=["NONE", "HOF"])

In [ ]:
train_vectors = vectorizer.fit_transform(train.post)
test_vectors = vectorizer.transform(test.post)

In [ ]:
train_vectors.shape

In [ ]:
print(c.predict_proba([test.iloc[0]["post"]]))